### Factorization Machines classifier

In [ ]:
#importing relevant libraries
from pyspark.sql import SparkSession
from pyspark.ml.classification import FMClassifier
from pyspark.ml.feature import MinMaxScaler, StringIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
#Creating sesion and loading data
spark = SparkSession.builder.appName("Factorization Machine classifier").getOrCreate()
data = spark.read.csv("Data\iris.csv", header = True, inferSchema = True)
data.show()

In [ ]:
data.columns

In [ ]:
#ML libraries 
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
featureasembler = VectorAssembler(inputCols = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width'], outputCol = 'features')
output = featureasembler.transform(data)
output.show()


In [ ]:
from pyspark.ml.feature import StringIndexer

labelIndexer = StringIndexer(inputCol = "class", outputCol = "indexedLabel").fit(output)
#Scale Features
featuresScaler = MinMaxScaler(inputCol = "features", outputCol = "scaledFeatures").fit(output)

In [ ]:
#Split the Data
train, test = output.randomSplit([0.6, 0.2], 1234)
print("Train Dataset: ", + train.count())
print("Test Dataset: ", + test.count())

In [ ]:
#Create pipeline
fm = FMClassifier(labelCol = "indexedLabel", featuresCol = "scaledFeatures", stepSize = 0.001)
pipeline =Pipeline(stages = [labelIndexer, featuresScaler, fm])

In [ ]:
#train Model
model = pipeline.fit(train)

In [ ]:
#Prediction
predictions = model.transform(test)

In [ ]:
predictions.select("prediction", "indexedLabel", "features").show(10)

In [ ]:
#Evaluate model
evaluator = MulticlassClassificationEvaluator(labelCol = "indexedLabel", predictionCol = "prediction", metric = "accuracy")
accuracy = evaluator.evaluate(predictions)
print("Accuracy: ", str(accuracy))

In [ ]:
#Print Relevant summary
fmModel = model.summary
print("Factor: ", + str(fmModel.factors))
print("Linear: ", str(fmModel.linear))
print("Intercept: ", str(fmModel.intercept))